In [ ]:
import os
import numpy as np
import sys
import panel as pn
import time
import json
import logging
import threading

from azure.eventhub import EventHubConsumerClient
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, Select
from bokeh.layouts import layout
from bokeh.plotting import figure
from datetime import datetime
from math import radians

pn.extension()
rng = np.random.default_rng(12345)

RECEIVED_MESSAGES = 0
connection_str = os.environ.get("IOTHUB_CONNECTION_STRING")
consumer_group = os.environ.get("IOTHUB_CONSUMER_GROUP")
eventhub_name= os.environ.get("IOTHUB_COMPATIBLE_NAME")

In [ ]:
# We produce new data here, and use the selector to discriminate
def message_handler(partition_context, event):
    logger.info("Received event from partition {}".format(partition_context.partition_id))

    message = event.body_as_json()

    print("\nMessage received:")
    print( "    Data: <<{}>>".format(message) )

    partition_context.update_checkpoint(event)


def on_error(partition_context, error):
    # Put your code here. partition_context can be None in the on_error callback.
    if partition_context:
        print("An exception: {} occurred during receiving from Partition: {}.".format(
            partition_context.partition_id,
            error
        ))
    else:
        print("An exception: {} occurred during the load balance process.".format(error))
        
        


In [ ]:
client = EventHubConsumerClient.from_connection_string(connection_str, 
                                                    consumer_group, 
                                                    eventhub_name=eventhub_name)
logger = logging.getLogger("azure.eventhub")
logging.basicConfig(level=logging.INFO)

with client:
    client.receive(
        on_event=message_handler,
        on_error=on_error,
        starting_position="-1",  # "-1" is from the beginning of the partition.
    )